In [1]:
## from urllib.request import urlretrieve
## urlretrieve("https://raw.githubusercontent.com/JasonFeng98/FinTech590-RiskManagement/main/Week04/Project/portfolio.csv", "Portfolio.csv")

In [2]:
## from urllib.request import urlretrieve
## urlretrieve("https://raw.githubusercontent.com/JasonFeng98/FinTech590-RiskManagement/main/Week04/DailyPrices.csv", "DailyPrices.csv")

# Question 1

In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import argparse
from numpy import linalg as LA
import scipy.stats as st
from scipy.optimize import minimize

In [4]:
## Simulate returns assuming it follows a normal distribution
mu, sigma = 0, 0.02
n = 15000
ret = np.random.normal(mu, sigma, n-1)
pd.DataFrame(ret)
p0 = 100

In [5]:
## Get the best fit distribution
def get_best_distribution(data):
    dist_names = ["norm", "lognorm"]
    dist_results = []
    params = {}
    for dist_name in dist_names:
        dist = getattr(st, dist_name)
        param = dist.fit(data)
        params[dist_name] = param
        D, p = st.kstest(data, dist_name, args=param)
        print("p value for "+dist_name+" = "+str(p))     
        dist_results.append((dist_name, p))
    best_dist, best_p = (max(dist_results, key=lambda item: item[1]))
    print("Best fitting distribution: "+str(best_dist))
    print("Best p value: "+ str(best_p))
    print("Parameters for the best fit: "+ str(params[best_dist]))

    return best_dist, params[best_dist]

In [6]:
## Classical Brownian Motion
## Simulation
price_c = ret + p0
pd.DataFrame(price_c)

,0
0,99.995802
1,99.954106
2,99.990604
3,100.035903
4,99.991815
...,...
14994,99.992452
14995,100.019321
14996,99.971180
14997,100.027562


In [7]:
par = get_best_distribution(price_c)[1]
par

p value for norm = 0.9358997260305307
p value for lognorm = 0.29153213823837754
Best fitting distribution: norm
Best p value: 0.9358997260305307
Parameters for the best fit: (100.00010495953047, 0.020105950277092718)


(100.00010495953047, 0.020105950277092718)

In [8]:
## Arithmetic Return
price_dis = (1 + ret) * p0
pd.DataFrame(price_dis)

,0
0,99.580215
1,95.410601
2,99.060412
3,103.590319
4,99.181545
...,...
14994,99.245191
14995,101.932125
14996,97.117964
14997,102.756162


In [9]:
par_dis = get_best_distribution(price_dis)[1]

p value for norm = 0.9358997260557549
p value for lognorm = 0.8000441908327963
Best fitting distribution: norm
Best p value: 0.9358997260557549
Parameters for the best fit: (100.01049595304646, 2.0105950277092783)


In [10]:
sigma_dis = sigma * p0
sigma_dis

2.0

In [11]:
## Geometric Return
price_geo = p0 * np.exp(ret)
pd.DataFrame(price_geo)

,0
0,99.581095
1,95.514321
2,99.064812
3,103.655550
4,99.184886
...,...
14994,99.248033
14995,101.950911
14996,97.159099
14997,102.794496


In [12]:
par_geo = get_best_distribution(price_geo)[1]

p value for norm = 0.26523183383597804
p value for lognorm = 0.9409585225592293
Best fitting distribution: lognorm
Best p value: 0.9409585225592293
Parameters for the best fit: (0.02100737961915651, 4.290616849994121, 95.71897346969942)


In [13]:
Mean_geo = np.log(par_geo[2])
sigma_geo = par_geo[0]
print(Mean_geo, sigma_geo)

4.5614165386794845 0.02100737961915651


In [14]:
exp_mean = np.log(p0)
exp_mean

4.605170185988092

# Question 2

In [15]:
## Return Calculation
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import argparse

def return_calculate(prices, method = None, dateColumn = "Date"):
    prices2 = np.matrix(prices.drop(columns = ['Date']))
    Vars = list(prices.columns.values)
    nVars = len(Vars)
    Vars.remove('Date')
    
    if nVars == len(Vars):
        print("DateColumn not in DataFrame")
        
    nVars = nVars - 1
    p = prices2
    n = p.shape[0]
    m = p.shape[1]
    p2 = np.empty([n-1,m], dtype=float)
    for i in range(0, n-1):
        for j in range(0, m):
            p2[i,j] = p[i+1,j] / p[i,j]
    
    if method.upper() == "DISCRETE":
         p2 = p2 - 1.0
    elif method.upper() == "LOG":
        p2 = math.log(p2)
    else:
        print("Method must be in (\"LOG\",\"DISCRETE\")")
    
    out = prices[[dateColumn]]
    out = out.iloc[1: , :]
    for i in range(0, nVars):
        out.loc[:, Vars[i]] = p2[:, i]
    return(out)

    

In [16]:
## Read the file and select the INTC
df1 = pd.read_csv('DailyPrices.csv')
df2 = df1[['Date','INTC']]

In [17]:
Return = return_calculate(df2, "DISCRETE", 'Date' )
Return

,Date,INTC
1,2021-10-21 00:00:00,0.011378
2,2021-10-22 00:00:00,-0.116786
3,2021-10-25 00:00:00,-0.001011
4,2021-10-26 00:00:00,-0.022870
5,2021-10-27 00:00:00,-0.008078
6,2021-10-28 00:00:00,0.003967
7,2021-10-29 00:00:00,0.019135
8,2021-11-01 00:00:00,0.011224
9,2021-11-02 00:00:00,0.006256
10,2021-11-03 00:00:00,0.010630


In [18]:
df3 = Return['INTC']
## Calculate the mean of stock price
Mean = df3.mean()
## Each lines minus the mean
INTC = df3 - Mean
## Calculate the standard deviation
STD = INTC.std()

In [19]:
# VaR with normal distribution
import scipy.stats as st
VaR_05 = st.norm.ppf(0.05, 0, STD)
VaR_01 = st.norm.ppf(0.01, 0, STD)
print(VaR_05, VaR_01)

-0.03443685422561183 -0.048704700104460985


In [20]:
n = df3.shape
n

(60,)

In [21]:
## VaR with normal distribution with exp weighted variance
w = np.empty(n, dtype = float)
## Calculate the mean for column
## Each x delete it stock return mean
x = INTC
n = INTC.shape[0]
lam = 0.94
# Calculate the weighted
for i in range (0,n):
    w[i] = (1 - lam)*lam**(n-i-1)
w = w / sum(w)
## Change w shape (60, ) to (60, 1)
w = w[:, np.newaxis]
## make x to array for elementwise multiple
x1 = np.array(x)
x1 = x1[:, np.newaxis]
res = w*x1

## Transpose
#res = res.T
## matrix multiplication
res = res * x1
Sum = sum(res)
Sum
STD2 = np.sqrt(Sum)
STD2

array([0.01588418])

In [22]:
## VaR with normal distribution with exp weighted variance
VaR_05 = st.norm.ppf(0.05, 0, STD2)
VaR_01 = st.norm.ppf(0.01, 0, STD2)
print(VaR_05, VaR_01)

[-0.02612715] [-0.03695213]


In [23]:
## VaR with MLE
Fit_return = st.t.fit(INTC)
VaR_05 = st.t.ppf(0.05,Fit_return[0], Fit_return[1], Fit_return[2])
VaR_01 = st.t.ppf(0.01, Fit_return[0], Fit_return[1], Fit_return[2])
print(VaR_05, VaR_01)

-0.026299935811070388 -0.04919266232015989


In [24]:
## VaR using Historical Simulation
Sorted_Return = sorted(INTC)
Arr_Return = np.array(Sorted_Return)
n_05 = 0.05 * Arr_Return.shape[0]
n_01 = 0.01 * Arr_Return.shape[0]
VaR_05 = Arr_Return[int(n_05)]
VaR_01 = Arr_Return[int(n_01)]
print(VaR_05, VaR_01)

-0.020589182603003648 -0.11722477150145412


In [25]:
## Empirical Distribution  of Return, in sample
Arr_INTC = np.array(INTC)
VaR_05 = np.quantile(Arr_INTC, 0.05)
VaR_01 = np.quantile(Arr_INTC, 0.01)
print(VaR_05, VaR_01)

-0.020725168575353364 -0.06280041914884281


In [26]:
## Empirical Distribution  of Return, out of sample
intcout = pd.read_csv("INTC.csv")
intcout = intcout[['Date','Close']]
rets = return_calculate(intcout, method="DISCRETE", dateColumn="Date")
rets.rename({"Close":"INTC"},axis=1, inplace=True)

x2 = np.asarray(rets['INTC'])
VaR_05_out = np.quantile(x2,0.05)
VaR_01_out = np.quantile(x2,0.01)
print(VaR_05_out, VaR_01_out)

-0.026318985805642858 -0.0585521908737391


# Problem 3

In [27]:
df4 = pd.read_csv('Portfolio.csv')
df4

,Portfolio,Stock,Holding
0,A,AAPL,58
1,A,TSLA,10
2,A,FB,31
3,A,JPM,64
4,A,HD,27
...,...,...,...
95,C,F,397
96,C,LRCX,14
97,C,MO,197
98,C,LMT,27


In [28]:
#Splitting dataframe
grouped = df4.groupby(df4.Portfolio)
df_A = grouped.get_group("A")
df_A = df_A.drop(columns = ['Portfolio'])
df_B = grouped.get_group("B")
df_B = df_B.drop(columns = ['Portfolio'])
df_C = grouped.get_group("C")
df_C = df_C.drop(columns = ['Portfolio'])
df_ALL = df4.drop(columns = ['Portfolio'])
holdingsA = dict(df_A.values)
holdingsB = dict(df_B.values)
holdingsC = dict(df_C.values)
holdingsALL = dict(df_ALL.values)

In [29]:
## Collect the Stock Key
df_A_arr = df_A.to_numpy()
df_B_arr = df_B.to_numpy()
df_C_arr = df_C.to_numpy()
df_ALL_arr = df_ALL.to_numpy()
Sym_A = df_A_arr[:,0]
Sym_B = df_B_arr[:,0]
Sym_C = df_C_arr[:,0]
Sym_ALL = df_ALL_arr[:,0]
Sym_ALL

array(['AAPL', 'TSLA', 'FB', 'JPM', 'HD', 'BAC', 'XOM', 'AVGO', 'PEP',
       'TMO', 'CMCSA', 'ACN', 'INTC', 'PYPL', 'MRK', 'T', 'LOW', 'NEE',
       'AMD', 'INTU', 'MS', 'AMAT', 'CVS', 'GS', 'BA', 'SBUX', 'GE',
       'ISRG', 'ANTM', 'MU', 'NOW', 'ADP', 'PNC', 'ADI', 'SYK', 'TJX',
       'MSFT', 'GOOGL', 'NVDA', 'JNJ', 'PG', 'MA', 'DIS', 'ADBE', 'KO',
       'NFLX', 'COST', 'WFC', 'WMT', 'LLY', 'NKE', 'LIN', 'UNP', 'UPS',
       'MDT', 'ORCL', 'RTX', 'AMGN', 'CAT', 'AMT', 'COP', 'AXP', 'SPGI',
       'BKNG', 'ZTS', 'MDLZ', 'GILD', 'GM', 'AMZN', 'GOOG', 'BRK-B',
       'UNH', 'V', 'PFE', 'CSCO', 'CVX', 'ABBV', 'ABT', 'CRM', 'VZ',
       'QCOM', 'MCD', 'DHR', 'TXN', 'PM', 'HON', 'BMY', 'SCHW', 'C',
       'BLK', 'IBM', 'PLD', 'TGT', 'DE', 'MMM', 'F', 'LRCX', 'MO', 'LMT',
       'TFC'], dtype=object)

In [30]:
# Filter prices and returns
Vars = list(df1.columns.values)
Vars.remove('Date')
Vars = np.array(Vars)
    
nm1 = np.intersect1d(Sym_A,Vars)
nm2 = np.intersect1d(Sym_B,Vars)
nm3 = np.intersect1d(Sym_C,Vars)
nmALL = np.intersect1d(Sym_ALL,Vars)

In [31]:
## Get Current Price for all Stock
current_prices = df1.iloc[-1:]
current_prices = current_prices.drop(columns = ['Date'])
current_prices

,SPY,AAPL,MSFT,AMZN,TSLA,GOOGL,GOOG,FB,NVDA,BRK-B,...,PNC,MDLZ,MO,ADI,GILD,LMT,SYK,GM,TFC,TJX
60,464.720001,173.070007,310.200012,3242.76001,1049.609985,2789.610107,2795.72998,331.899994,269.420013,324.130005,...,222.199997,68.099998,50.810001,172.0,71.769997,372.619995,266.709991,61.099998,67.410004,71.18


In [32]:
## Get Current price for the Portfolio stock
current_pricesA = current_prices[nm1]
current_pricesB = current_prices[nm2]
current_pricesC = current_prices[nm3]
current_pricesALL = current_prices[nmALL]

In [33]:
Return = return_calculate(df1, "DISCRETE", 'Date' )
Return

,Date,SPY,AAPL,MSFT,AMZN,TSLA,GOOGL,GOOG,FB,NVDA,...,PNC,MDLZ,MO,ADI,GILD,LMT,SYK,GM,TFC,TJX
1,2021-10-21 00:00:00,0.002608,0.001474,0.010897,0.005842,0.032571,0.000825,0.002566,0.003228,0.026648,...,-0.000329,-0.002156,-0.004130,0.010683,0.001944,0.005416,0.009789,0.012832,-0.004696,0.012841
2,2021-10-22 00:00:00,-0.001036,-0.005285,-0.005149,-0.028955,0.017539,-0.030443,-0.029104,-0.050515,0.001498,...,0.015748,0.005984,-0.000207,-0.002181,0.003732,0.003913,-0.002568,-0.010957,0.013526,-0.008040
3,2021-10-25 00:00:00,0.005363,-0.000336,-0.003332,-0.004551,0.126616,-0.000869,0.001068,0.012569,0.019361,...,0.002221,-0.002974,-0.005599,0.011770,0.006395,0.004618,-0.009720,-0.000173,0.002017,0.017145
4,2021-10-26 00:00:00,0.000900,0.004575,0.006426,0.016775,-0.006274,0.013543,0.006478,-0.039186,0.066952,...,-0.002263,0.008121,0.003337,-0.003545,0.000887,-0.118035,0.000476,-0.006752,-0.002013,0.001073
5,2021-10-27 00:00:00,-0.004430,-0.003148,0.042114,0.004864,0.019078,0.049595,0.048367,-0.011368,-0.010762,...,-0.014625,-0.007233,-0.012679,-0.052368,-0.014174,-0.002922,-0.017132,-0.054210,-0.022191,-0.011786
6,2021-10-28 00:00:00,0.009649,0.024992,0.003651,0.015941,0.037751,-0.002520,-0.002039,0.015054,0.020040,...,0.000658,0.009273,-0.061474,0.011733,0.008537,0.001360,-0.000223,-0.000369,0.013966,0.009913
7,2021-10-29 00:00:00,0.002029,-0.018156,0.022414,-0.021511,0.034316,0.015064,0.014655,0.020983,0.025099,...,-0.009481,-0.003445,-0.010543,0.005972,-0.036531,0.002806,-0.008829,0.003503,-0.006574,0.004448
8,2021-11-01 00:00:00,0.001720,-0.005607,-0.006785,-0.016107,0.084910,-0.030727,-0.030326,0.019810,0.010169,...,-0.001848,0.002799,-0.001360,0.008646,0.011714,-0.004183,0.008757,0.019475,0.015125,0.017713
9,2021-11-02 00:00:00,0.004043,0.007116,0.011416,-0.001615,-0.030275,0.013488,0.014530,-0.005758,0.022225,...,-0.002564,0.007388,0.002270,0.016858,0.003961,-0.013961,0.015835,0.001802,-0.002018,0.010053
10,2021-11-03 00:00:00,0.006105,0.009799,0.002611,0.021508,0.035717,0.008014,0.006355,0.010790,0.007462,...,0.012375,0.015971,0.018347,0.004833,0.022914,0.015445,-0.002898,0.007375,0.013064,0.016488


In [34]:
## Calculate Return
ReturnA = Return[nm1]
ReturnB = Return[nm2]
ReturnC = Return[nm3]
ReturnAll = Return[nmALL]

In [35]:
# Calculate portfolio VAR for A
from numpy.linalg import eigh
pvA = 0
deltaA = np.empty(len(nm1), dtype = float)
deltaB = np.empty(len(nm2), dtype = float)
deltaC = np.empty(len(nm3), dtype = float)
deltaALL = np.empty(len(nmALL), dtype = float)
i = 0
for s in nm1:
    value1 = holdingsA[s] * current_prices[s]
    pvA += value1
    deltaA[i] = value1
    i += 1
    
deltaA = deltaA[:, np.newaxis]
deltaA = np.divide(deltaA,float(pvA))
Sigma = ReturnA.cov()
e = eigh(Sigma)
p_sig = np.sqrt(deltaA.T @ Sigma @ deltaA)
VaRA = -pvA * st.norm.ppf(0.05) * p_sig.iloc[0,0]
VaRA

60    6003.221298
Name: AAPL, dtype: float64

In [36]:
# Calculate portfolio VaR for B
from numpy.linalg import eigh
pvB = 0
i = 0
for s in nm2:
    value2 = holdingsB[s] * current_prices[s]
    pvB += value2
    deltaB[i] = value2
    i += 1
    
deltaB = deltaB[:, np.newaxis]
deltaB = np.divide(deltaB,float(pvB))
Sigma = ReturnB.cov()
e = eigh(Sigma)
p_sig = np.sqrt(deltaB.T @ Sigma @ deltaB)
VaRB = -pvB * st.norm.ppf(0.05) * p_sig.iloc[0,0]
VaRB

60    4886.596042
Name: ADBE, dtype: float64

In [37]:
# Calculate portfolio VaR for C
from numpy.linalg import eigh
pvC = 0
i = 0
for s in nm3:
    value3 = holdingsC[s] * current_prices[s]
    pvC += value3
    deltaC[i] = value3
    i += 1
    
deltaC = deltaC[:, np.newaxis]
deltaC = np.divide(deltaC,float(pvC))
Sigma = ReturnC.cov()
e = eigh(Sigma)
p_sig = np.sqrt(deltaC.T @ Sigma @ deltaC)
VaRC = -pvC * st.norm.ppf(0.05) * p_sig.iloc[0,0]
VaRC

60    3679.556069
Name: ABBV, dtype: float64

In [38]:
# Calculate portfolio VaR for Total
from numpy.linalg import eigh
pvALL = 0
i = 0
for s in nmALL:
    value4 = holdingsALL[s] * current_prices[s]
    pvALL += value4
    deltaALL[i] = value4
    i += 1
    
deltaALL = deltaALL[:, np.newaxis]
deltaALL = np.divide(deltaALL,float(pvALL))
Sigma = ReturnAll.cov()
e = eigh(Sigma)
p_sig = np.sqrt(deltaALL.T @ Sigma @ deltaALL)
VaRALL = -pvALL * st.norm.ppf(0.05) * p_sig.iloc[0,0]
VaRALL

60    14100.550125
Name: AAPL, dtype: float64

In [39]:
## MC VaR
n = 10000
mu_A = np.zeros(len(nm1))
mu_B = np.zeros(len(nm2))
mu_C = np.zeros(len(nm3))
mu_ALL = np.zeros(len(nmALL))

SigmaA = np.cov(ReturnA.T)
SigmaB = np.cov(ReturnB.T)
SigmaC = np.cov(ReturnC.T)
SigmaALL = np.cov(ReturnAll.T)

SimreturnA = np.random.multivariate_normal(mu_A, SigmaA, n)
SimreturnB = np.random.multivariate_normal(mu_B, SigmaB, n)
SimreturnC = np.random.multivariate_normal(mu_C, SigmaC, n)
SimreturnALL = np.random.multivariate_normal(mu_ALL, SigmaALL, n)

In [40]:
## Get Prices
SimPricesA = (1 + SimreturnA) * np.array(current_pricesA).ravel().T
SimPricesB = (1 + SimreturnB) * np.array(current_pricesB).ravel().T
SimPricesC = (1 + SimreturnC) * np.array(current_pricesC).ravel().T
SimPricesALL = (1 + SimreturnALL) * np.array(current_pricesALL).ravel().T

In [41]:
## MC Normal VaR Function
def mc_calculate(holdings, names, sim_prices,n, PV):
    vHoldings = np.array([holdings[s] for s in names])
    pVals = sim_prices @ vHoldings
    pVals = np.array(pVals).ravel()
    pVals.sort()
    a=int(0.05*n)
    VaR = PV - pVals[a]
    
    return VaR

In [42]:
## MC Normal VaR
mcA = mc_calculate(holdingsA, nm1, SimPricesA, n, pvA)
mcB = mc_calculate(holdingsB, nm2, SimPricesB, n, pvB)
mcC = mc_calculate(holdingsC, nm3, SimPricesC, n, pvC)
mcALL = mc_calculate(holdingsALL, nmALL, SimPricesALL, n, pvALL)
print(mcA)
print(mcB)
print(mcC)
print(mcALL)

60    6031.353035
Name: AAPL, dtype: float64
60    4713.121717
Name: ADBE, dtype: float64
60    3678.559149
Name: ABBV, dtype: float64
60    14159.42006
Name: AAPL, dtype: float64


In [43]:
def pvals(holdings, names, sim_prices):
    vHoldings = np.array([holdings[s] for s in names])
    pVals = sim_prices@vHoldings
    pVals = np.array(pVals).ravel()
    pVals.sort()
        
    return pVals

In [44]:
# Gaussian kernal density estimate
from scipy.stats import gaussian_kde

mcpvalsA = pvals(holdingsA, nm1, SimPricesA)
mcpvalsB = pvals(holdingsB, nm2, SimPricesB)
mcpvalsC = pvals(holdingsC, nm3, SimPricesC)
mcpvalsALL = pvals(holdingsALL, nmALL, SimPricesALL)

kdemcA = gaussian_kde(mcpvalsA).resample(size=n)
kdemcB = gaussian_kde(mcpvalsB).resample(size=n)
kdemcC = gaussian_kde(mcpvalsC).resample(size=n)
kdemcALL = gaussian_kde(mcpvalsALL).resample(size=n)

In [45]:
## VaR 
VaRmcKDEA = pvA-np.quantile(kdemcA, 0.05)
VaRmcKDEB = pvB-np.quantile(kdemcB, 0.05)
VaRmcKDEC = pvC-np.quantile(kdemcC, 0.05)
VaRmcKDEALL = pvALL-np.quantile(kdemcALL, 0.05)
print(VaRmcKDEA)
print(VaRmcKDEB)
print(VaRmcKDEC)
print(VaRmcKDEALL)

60    6133.180296
Name: AAPL, dtype: float64
60    4877.027258
Name: ADBE, dtype: float64
60    3809.341859
Name: ABBV, dtype: float64
60    14209.990069
Name: AAPL, dtype: float64


In [46]:
## Historical VaR
#Simulate prices, sample returns from history
simpricesAhist = (1 + ReturnA) * np.array(current_pricesA).ravel().T
simpricesBhist = (1 + ReturnB) * np.array(current_pricesB).ravel().T
simpricesChist = (1 + ReturnC) * np.array(current_pricesC).ravel().T
simpricesALLhist = (1 + ReturnAll) * np.array(current_pricesALL).ravel().T

In [47]:
def histVaR_calculate(holdings, names, sim_prices, PV, returns):
    vHoldings = np.array([holdings[s] for s in names])
    pVals = sim_prices@vHoldings
    pVals = np.array(pVals).ravel()
    pVals.sort()
    n = returns.shape[0]
    a=int(0.05*n)
    VaR = PV - pVals[a]
    return VaR

In [53]:
histA = histVaR_calculate(holdingsA, nm1, simpricesAhist, pvA, ReturnA)
histB = histVaR_calculate(holdingsB, nm2, simpricesBhist, pvB, ReturnB)
histC = histVaR_calculate(holdingsC, nm3, simpricesChist, pvC, ReturnC)
histALL = histVaR_calculate(holdingsALL, nmALL, simpricesALLhist, pvALL, ReturnAll)
print(histA)
print(histB)
print(histC)
print(histALL)

60    5298.490899
Name: AAPL, dtype: float64
60    5576.130248
Name: ADBE, dtype: float64
60    3307.758233
Name: ABBV, dtype: float64
60    12460.873754
Name: AAPL, dtype: float64


In [49]:
## Gaussian kernal density
histpvalsA = pvals(holdingsA, nm1, simpricesAhist)
histpvalsB = pvals(holdingsB, nm2, simpricesBhist)
histpvalsC = pvals(holdingsC, nm3, simpricesChist)
histpvalsALL = pvals(holdingsALL, nmALL, simpricesALLhist)

kdehistA = gaussian_kde(histpvalsA).resample(size=n)
kdehistB = gaussian_kde(histpvalsB).resample(size=n)
kdehistC = gaussian_kde(histpvalsC).resample(size=n)
kdehistALL = gaussian_kde(histpvalsALL).resample(size=n)

In [50]:
##VaR
VaRhistKDEA = pvA-np.quantile(kdehistA, 0.05)
VaRhistKDEB = pvB-np.quantile(kdehistB, 0.05)
VaRhistKDEC = pvC-np.quantile(kdehistC, 0.05)
VaRhistKDEALL = pvALL-np.quantile(kdehistALL, 0.05)
print(VaRhistKDEA)
print(VaRhistKDEB)
print(VaRhistKDEC)
print(VaRhistKDEALL)

60    6607.530596
Name: AAPL, dtype: float64
60    5948.630563
Name: ADBE, dtype: float64
60    3987.124178
Name: ABBV, dtype: float64
60    15411.866759
Name: AAPL, dtype: float64


In [51]:
print(pvA)
print(pvB)
print(pvC)
print(pvALL)

60    364532.96043
Name: AAPL, dtype: float64
60    326770.148861
Name: ADBE, dtype: float64
60    326727.670717
Name: ABBV, dtype: float64
60    1.018031e+06
Name: AAPL, dtype: float64
